二叉堆在逻辑上其实是一种特殊的二叉树（完全二叉树），只不过存储在数组里。一般的链表二叉树，我们操作节点的指针，而在数组里，我们把数组索引作为指针，

由于二叉堆是一个完全二叉树，因此通过数组索引，我们可以很方便的得到目标节点的父结点（i // 2）、左孩子节点（i * 2）、右孩子节点（i * 2 + 1）

二叉堆还分为最大堆和最小堆。**最大堆的性质是：每个节点都大于等于它的两个子节点**。类似的，最小堆的性质是：每个节点都小于等于它的子节点

## 优先级队列

优先级队列这种数据结构有一个很有用的功能，你插入或者删除元素的时候，元素会自动排序，这底层的原理就是二叉堆的操作。

优先级队列有两个主要 API，分别是 `insert` 插入一个元素和 `delMax` 删除最大元素（如果底层用最小堆，那么就是 `delMin`）。

关于 swim 和 sink，我们要讲的是最大堆，每个节点都比它的两个子节点大，但是在插入元素和删除元素时，难免破坏堆的性质，这就需要通过这两个操作来恢复堆的性质了。

对于最大堆，会破坏堆性质的有两种情况：

1、如果某个节点 A 比它的子节点（中的一个）小，那么 A 就不配做父节点，应该下去，下面那个更大的节点上来做父节点，这就是对 A 进行**下沉**。

2、如果某个节点 A 比它的父节点大，那么 A 不应该做子节点，应该把父节点换下来，自己去做父节点，这就是对 A 的**上浮**。

当然，错位的节点 A 可能要上浮（或下沉）很多次，才能到达正确的位置，恢复堆的性质。所以代码中肯定有一个 `while` 循环。

但是这两个操作不是互逆吗，所以上浮的操作一定能用下沉来完成，为什么我还要费劲写两个方法？

是的，操作是互逆等价的，但是最终我们的操作只会在堆底和堆顶进行（等会讲原因），显然堆底的「错位」元素需要上浮，堆顶的「错位」元素需要下沉。

In [ ]:
class MaxHeap:
    def __init__(self):
        # 存储元素的数组
        self.pq = [None]
        # 当前 Priority Queue 中的元素个数
        self.size = 0

    def max(self):
        """返回当前队列中最大元素
        索引0是不用的因此返回的1
        从索引1开始是为了计算节点方便
        """
        return self.pq[1]

    def insert(self, e):
        """插入元素 e"""
        self.pq.append(e)
        self.size += 1
        # 上浮到正确的位置
        self.__swim(self.size)

    def delMax(self):
        """删除并返回当前队列中最大元素"""
        # 把这个最大元素换到最后
        self.__swap(1, self.size)
        # 删除最大的元素
        max = self.pq.pop()
        self.size -= 1
        # 让 pq[1] 下沉到正确位置
        self.__sink(1)
        return max

    def __swim(self, x):
        """上浮第 x 个元素，以维护最大堆性质"""
        while x > 1 and self.pq[self.__parent(x)] < self.pq[x]:
            self.__swap(self.__parent(x), x)
            x = self.__parent(x)

    def __sink(self, x):
        """下沉第 x 个元素，以维护最大堆性质
        下沉比上浮略微复杂一点，因为上浮某个节点 x
        只需要 x 和其父节点比较大小即可；但是下沉某个节点 x
        需要 x 和其两个子节点比较大小，如果 x 不是最大的就需要调整位置，要把较大的那个子节点和 x 交换。
        """
        while self.__left(x) <= self.size:
            max = self.__left(x)
            if self.__right(x) <= self.size and self.pq[max] < self.pq[self.__right(x)]:
                max = self.__right(x)
            if self.pq[x] >= self.pq[max]:
                break
            self.__swap(max, x)
            x = max

    def __swap(self, i, j):
        """交换数组的两个元素"""
        self.pq[i], self.pq[j] = self.pq[j], self.pq[i]

    def __parent(self, i):
        """返回i的父结点"""
        return i // 2

    def __left(self, i):
        """返回i的左孩子结点"""
        return i * 2

    def __right(self, i):
        """返回i的右孩子结点"""
        return i * 2 + 1

    def __str__(self):
        return str(self.pq)


max_heap = MaxHeap()
max_heap.insert(10)
max_heap.insert(20)
max_heap.insert(5)
max_heap.insert(30)
print(max_heap)  # Output: [30, 20, 5, 10]
print(max_heap.delMax())  # Output: 30
print(max_heap)  # Output: [20, 10, 5]

[None, 30, 20, 5, 10]
30
[None, 20, 10, 5]


至此，一个优先级队列就实现了，插入和删除元素的时间复杂度为 `O(logK)`，`K` 为当前二叉堆（优先级队列）中的元素总数。因为我们时间复杂度主要花费在 `sink` 或者 `swim` 上，而不管上浮还是下沉，最多也就树（堆）的高度，也就是 log 级别。